In [1]:
#Importing the same libraries as earlier
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
import spacy
import textblob
import seaborn as sns

from collections import Counter, defaultdict
import time
from datetime import datetime
import re
import math

import warnings
warnings.filterwarnings('ignore')

In [2]:
#Downloading the relevant libraries and dependencies in NLTK module for preprocessing
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [3]:
#Here we will import the processed dataset for demonstration
data = pd.read_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/processed_data.csv')

In [4]:
data.head()

,Unnamed: 0,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len
0,0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close 14 6 per cent 35 000 240 000 ATMs India ...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",10,1,2,2,0.041667,25,23,143,0,1,4.760000
1,1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director chief executi...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,24,24,156,1,4,5.541667
2,2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor wife also charged abetting crime,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,3,0.000000,15,15,100,1,3,5.733333
3,3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said gearing extend stimulus package ...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",16,0,1,3,0.000000,23,23,157,0,7,5.869565
4,4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit financial year ended March 31 2020 ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,1,1,2,0.160000,30,24,149,2,5,4.000000


In [5]:
#Again dropping the unnanmed column as before
data.drop(['Unnamed: 0'], axis=1, inplace=True)

In [6]:
data.shape

(11995, 19)

In [7]:
#Firstly, we will spligt the dataset into training and test sets 
data_train = data[ : 10000]
data_test = data[10000 : ]

In [8]:
#I have to maintain a count of the words in vocab for applying the Turney's algorithm for our dataset
#I will be maintaing a Python dictionary where the corpus' tokens will be keys and their corresponding count values

def def_value():
  return "Not Present"

vocab_dict = defaultdict(def_value)

def cnt_vocab(messages):

  for i, message in enumerate(messages):
    messages[i] = nltk.word_tokenize(messages[i])
    text = messages[i]

    for word in text:
      if vocab_dict[word] == "Not Present":
        vocab_dict[word] = 1
      else:
        vocab_dict[word] += 1
  return vocab_dict

In [9]:
#Here we define a dictionary data structure which acts as a frequency Mapper for all tokens in the corpus
messages, token_dict = [], defaultdict(def_value)
for i in range(len(data_train['Combined_Text'])):
  text = data_train['Combined_Text'][i]
  messages.append(text)

token_dict = cnt_vocab(messages)

**Turney's Thumbs-Up or Thumbs-Down algorithm:**


Turney's Thumbs-Up or Thumbs-Down algorithm is stated below:
1. First a POS(Parts of Speech) tagger is applied to the review. For each review, two consecutive words are extracted if their tags conform to any of the patterns in the manner as would be described by the rules as stated in the function.
2. Second step is to estimate the semantic orientation of the extracted phrases, using PMI-IR algorithm. Estimate the semantic orientation of the extracted phrases, using the PMI-IR algorithm. Uses mutual information as a strength of semantic orientation between two words.
3. The third step is to calculate the average semantic orientation of the phrases in the given review and classify the review as recommended if the average is positive and otherwise not recommended.



1st step implementation - Pattern-based POS tag extraction


In [10]:
'''
  elif (postag[k][1]=='VB') and (postag[k+1][1]=='NN'):  #new rule added
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

  elif (postag[k][1]=='VBN') and (postag[k+1][1]=='NN'):  #new rule added
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))
'''

def find_pattern(row):
  text = str(row['Combined_Text'])
  postag = nltk.pos_tag(nltk.word_tokenize(text))
  tag_pattern = []

  for k in range(0, len(postag)-2):
    if (postag[k][1]=='JJ' and (postag[k+1][1]=='NN' or postag[k+1][1]=='NNS')):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='RB' or postag[k][1]=='RBR' or postag[k][1]=='RBS') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='JJ') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN') and (postag[k+1][1]=='VBG'): #example - profit declines
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN') and (postag[k+1][1]=='VBD'): #example - profit declined
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='VBG') and (postag[k+1][1]=='NN'): #example - company grows
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='VBD') and (postag[k+1][1]=='NN'): #example - company dropped
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='JJS') and (postag[k+1][1]=='NN'): #example - top loser
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='NN' or postag[k][1]=='NNS') and (postag[k+1][1]=='JJ') and (postag[k+2][1]!='NN' or postag[k+2][1]!='NNS'):
       tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

    elif (postag[k][1]=='RB' or postag[k][1]=='RBR' or postag[k][1] == 'RBS') and (postag[k+1][1]=='VB' or postag[k+1][1]=='VBD' or postag[k+1][1]=='VBN' or postag[k+1][1]=='VBG'):
      tag_pattern.append("".join(postag[k][0]) + " " + "".join(postag[k+1][0]))

  return tag_pattern

In [11]:
data_train['Phrases'] = data_train.apply(lambda row : find_pattern(row), axis=1)

In [12]:
total_text = []
for row in range(len(data_train.index)):
  total_text.append(' '.join(str(text) for text in data_train.loc[row, ['Combined_Text']]))

2nd step - Capture the semantic orientation of extracted phrases, using PMI-IR algorithm

In [13]:
#How to capture the polarity of a phrase? Positive phrases co-occur more with "excellent", whereas Negative phrases co-occur more with "poor"

'''
NEAR operator has been derived from the following resource:
https://github.com/andelacolak/Turney-algorithm---sentiment-analysis/blob/master/turney_algorithm.py

'''
def near_operator(phrase, word, text_total):

  try:
    #In depth study about NEAR operator is required
    string= word+r'\W+(?:\w+\W+){0, 500}?'+phrase+r'|'+phrase+r'\W+(?:\w+\W+){0,500}?'+word
    #Try out different ways and different string patterns

    freq_words_in_range = len(re.findall(string, text_total))
    return freq_words_in_range

  except:
    return 0


In [14]:
def pos_phrase_hits(row, text_total):
  phrases = row['Phrases']
  phrases_length = len(row['Phrases'])
  pos_phrase_hits = [0.01] * phrases_length

 
  for ind, phrase in enumerate(phrases):
    pos_phrase_hits[ind] = near_operator(phrase, "boost", text_total)
    
    #pos_hits += text_total.count("boost")

  return pos_phrase_hits

In [15]:
def neg_phrase_hits(row, text_total):
  phrases = row['Phrases']
  phrases_length = len(row['Phrases'])
  neg_phrase_hits = [0.01] * phrases_length

  for ind, phrase in enumerate(phrases):
    neg_phrase_hits[ind] = near_operator(phrase, "decline", text_total)
    
    #neg_hits += text_total.count("decline")

  return neg_phrase_hits

In [16]:
text_total = '.'.join(total_text)

In [17]:
data_train['Positive_Hits'] = data_train.apply(lambda row : pos_phrase_hits(row, text_total), axis=1)

In [18]:
#data_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/turney1.csv')

In [19]:
data_train['Negative_Hits'] = data_train.apply(lambda row : neg_phrase_hits(row, text_total), axis=1)

In [20]:
#A day in the life of this tuple-based lists
life = [('Good Boy', 'NN'), ('Bad Boy', 'JJ')]
life[0][1]

'NN'

In [21]:
data_train['Positive_Hits'][4000]

[0, 1, 25, 0, 0, 4]

In [22]:
data_train['Phrases'][4]

['profit financial', 'financial year']

In [23]:
print(data_train.shape)
print(data_test.shape)

(10000, 22)
(1995, 19)


In [24]:
data_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/calculate_pmi.csv')

**3rd Step - Calcualting PMI**

In [25]:
t_train = pd.read_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/calculate_pmi.csv')

In [26]:
t_train.drop(['Unnamed: 0'], axis=1, inplace=True)

In [27]:
t_train.shape

(10000, 22)

In [28]:
def total_pos_hits(row, pos_hits, neg_hits):
  phrases = row['Phrases']
  pos_hits += (text_total.count("boost") * len(phrases))
  return pos_hits

In [29]:
def total_neg_hits(row, pos_hits, neg_hits):
  phrases = row['Phrases']
  neg_hits += (text_total.count("decline") * len(phrases))
  return neg_hits

In [30]:
#Define the threshold for positive as well as negative hits. One can hyperparameterzize and fine-tune it as per one's requirements
pos_hits = 0.01
neg_hits = 0.01

In [31]:
final_pos_hits = t_train.apply(lambda row : total_pos_hits(row, pos_hits, neg_hits), axis=1)
final_neg_hits = t_train.apply(lambda row : total_neg_hits(row, pos_hits, neg_hits), axis=1)

In [32]:
final_pos_hits = np.round(sum(final_pos_hits))
final_neg_hits = np.round(sum(final_neg_hits))

In [33]:
print('The number of positive hits are {} and the number of negative hits are {}'.format(final_pos_hits, final_neg_hits))

The number of positive hits are 72217476.0 and the number of negative hits are 66414294.0


In [34]:
type(final_pos_hits)

numpy.float64

In [35]:
regex = '\d+' 
re.findall(regex, t_train['Positive_Hits'][0])

['1', '1', '3']

In [36]:
def calculate_sentiment(row, final_pos_hits, final_neg_hits):
  phrases = list(row['Phrases'])
  pos_hit_list, neg_hit_list = [], []
  regex = '\d+' 

  pos_hit_list = re.findall(regex, row['Positive_Hits'])
  neg_hit_list = re.findall(regex, row['Negative_Hits'])

  #Now we will be calcualting the average semantic orientation of each phrase
  #We will predefine the average semantic orientation as 0.0
  
  avg_so = 0.0

  try:
    if (len(pos_hit_list)!=0 and len(neg_hit_list)!=0):
      so = [0] * len(pos_hit_list)

      for i in range(len(pos_hit_list)):

        if (pos_hit_list[i] != 0 and neg_hit_list[i] != 0):
          so[i] = np.log2((float(pos_hit_list[i]) * float(final_neg_hits)) / (float(neg_hit_list[i]) * float(final_pos_hits)))

        else:
          so[i] = 0

      avg_so = sum(so)/len(so)

      if np.round(avg_so) > 0.0:
        avg_so = 1.0

  except Exception as e:
    print(str(e))
      
  return avg_so

In [37]:
t_train['Label'] = t_train.apply(lambda row : calculate_sentiment(row, final_pos_hits, final_neg_hits), axis=1)

Streaming output truncated to the last 5000 lines.
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float division by zero
float 

In [38]:
t_train.head(n=10)

,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len,Phrases,Positive_Hits,Negative_Hits,Label
0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close 14 6 per cent 35 000 240 000 ATMs India ...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",10,1,2,2,0.041667,25,23,143,0,1,4.760000,"['virtual bank', 'instant creditclose', 'new a...","[1, 1, 3]","[1, 1, 4]",-0.259200
1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director chief executi...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,24,24,156,1,4,5.541667,"['senior forgo', 'managing director', 'directo...","[1, 5, 2, 10, 1]","[1, 11, 4, 8, 1]",-0.483969
2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor wife also charged abetting crime,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,3,0.000000,15,15,100,1,3,5.733333,"['huge scam', 'many year', 'also charged']","[1, 1, 1]","[1, 2, 1]",-0.454188
3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said gearing extend stimulus package ...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",16,0,1,3,0.000000,23,23,157,0,7,5.869565,"['bank said', 'gearing extend', 'package annou...","[11, 1, 1]","[11, 1, 1]",-0.120854
4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit financial year ended March 31 2020 ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,1,1,2,0.160000,30,24,149,2,5,4.000000,"['profit financial', 'financial year']","[1, 22]","[0, 22]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
65,2020-05-04,"Allow restructuring of loans, ask raters not t...",Mutual Funds say they didn't need to use their...,2020,Allow restructuring loan ask raters downgrade ...,mutual Funds say need use liquidity line fully,Allow restructuring loan ask raters downgrade ...,"[('Allow', 'NNP'), ('restructuring', 'NN'), ('...",11,1,0,3,0.000000,15,15,101,0,2,5.800000,[],[],[],0.000000
66,2020-05-03,As banker it is my first job to be safe: IDFC ...,"From now on, the system should open up systema...",2020,as banker first job safe IDFC First Bank CEO V...,from system open systematically people financi...,as banker first job safe IDFC First Bank CEO V...,"[('as', 'IN'), ('banker', 'NN'), ('first', 'JJ...",11,2,4,2,0.000000,21,21,137,3,4,5.571429,"['banker first', 'first job', 'job safe', 'sys...","[1, 1, 1, 1, 1, 1]","[0, 0, 0, 0, 0, 0]",0.000000
67,2020-05-03,More than 90% CKP Co-op Bank depositors to get...,The RBI cancelled the licence of CKP Co-op Ban...,2020,more 90 CKP Co op Bank depositor get back mone...,the RBI cancelled licence CKP Co op Bank April...,more 90 CKP Co op Bank depositor get back mone...,"[('more', 'RBR'), ('90', 'CD'), ('CKP', 'NNP')...",15,1,1,5,0.227273,26,22,150,3,6,4.807692,"['back money', 'cancelled licence', 'notified ...","[1, 1, 1]","[0, 0, 0]",0.000000
68,2020-05-03,"BoB NPAs surge six-fold, Indian Bank sees four...","The number o

In [39]:
def format_label(row):
  avg_so = row['Label']

  if avg_so < -0.01:
    avg_so = 0
  else:
    avg_so = 1

  return avg_so

In [40]:
t_train.head(80)

,Date,Title,Description,Year,Clean_Title,Clean_Desc,Combined_Text,Combined_Tag_Tokens,Title_Noun_Count,Title_Adverb_Count,Title_Adjective_Count,Title_Verb_Count,Jaccard_Index,num_words,num_unique_words,num_chars,num_words_upper,num_words_title,mean_word_len,Phrases,Positive_Hits,Negative_Hits,Label
0,2020-05-26,"ATMs to become virtual bank branches, accept d...","Close to 14.6 per cent (or 35,000) of the 240,...",2020,ATMs become virtual bank branch accept deposit...,close 14 6 per cent 35 000 240 000 ATMs India ...,ATMs become virtual bank branch accept deposit...,"[('ATMs', 'NNP'), ('become', 'VBP'), ('virtual...",10,1,2,2,0.041667,25,23,143,0,1,4.760000,"['virtual bank', 'instant creditclose', 'new a...","[1, 1, 3]","[1, 1, 4]",-0.259200
1,2020-05-26,IDFC First Bank seniors to forgo 65% of bonus ...,"V Vaidyanathan, managing director and chief ex...",2020,IDFC First Bank senior forgo 65 bonus amid Cov...,V Vaidyanathan managing director chief executi...,IDFC First Bank senior forgo 65 bonus amid Cov...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",13,0,2,4,0.000000,24,24,156,1,4,5.541667,"['senior forgo', 'managing director', 'directo...","[1, 5, 2, 10, 1]","[1, 11, 4, 8, 1]",-0.483969
2,2020-05-25,"Huge scam in YES Bank for many years, says Enf...",Rana Kapoor's wife also charged with abetting ...,2020,huge scam YES Bank many year say Enforcement D...,Rana Kapoor wife also charged abetting crime,huge scam YES Bank many year say Enforcement D...,"[('huge', 'JJ'), ('scam', 'NNS'), ('YES', 'NNP...",9,1,2,3,0.000000,15,15,100,1,3,5.733333,"['huge scam', 'many year', 'also charged']","[1, 1, 1]","[1, 2, 1]",-0.454188
3,2020-05-24,"Bank of Maharashtra sanctioned Rs 2,789 cr in ...",The bank said it was now gearing up to extend ...,2020,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,the bank said gearing extend stimulus package ...,Bank Maharashtra sanctioned Rs 2 789 cr loan M...,"[('Bank', 'NNP'), ('Maharashtra', 'NNP'), ('sa...",16,0,1,3,0.000000,23,23,157,0,7,5.869565,"['bank said', 'gearing extend', 'package annou...","[11, 1, 1]","[11, 1, 1]",-0.120854
4,2020-05-23,DCB Bank's profit before tax declines 37.6% to...,Net profit for the financial year ended March ...,2020,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,net profit financial year ended March 31 2020 ...,DCB Bank profit tax decline 37 6 Rs 93 84 cror...,"[('DCB', 'NNP'), ('Bank', 'NNP'), ('profit', '...",16,1,1,2,0.160000,30,24,149,2,5,4.000000,"['profit financial', 'financial year']","[1, 22]","[0, 22]",0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
75,2020-05-02,"Discretionary loans, cross-selling likely to r...",Larger challenge is to communicate to borrower...,2020,Discretionary loan cross selling likely reduce...,Larger challenge communicate borrower differen...,Discretionary loan cross selling likely reduce...,"[('Discretionary', 'NNP'), ('loan', 'NN'), ('c...",9,0,2,3,0.066667,15,15,134,0,1,8.000000,['cross selling'],[2],[2],-0.120854
76,2020-05-01,"IDFC First Bank to boost capital by Rs 2,000 c...",Capital infusion to be supported by IDFC limit...,2020,IDFC First Bank boost capital Rs 2 000 cr pref...,Capital infusion supported IDFC limited ICICI ...,IDFC First Bank boost capital Rs 2 000 cr pref...,"[('IDFC', 'NNP'), ('First', 'NNP'), ('Bank', '...",17,0,2,3,0.043478,24,22,160,4,10,5.708333,"['cr preferential', 'preferential issueCapital...","[0, 0, 0, 0]","[0, 0, 0, 0]",0.000000
77,2020-05-01,Regulatory benefits for banks helping mutual f...,"On April 27, the RBI had introduced the Rs 50,...",2020,Regulatory benefit bank helping mutual fund in...,on April 27 RBI introduced Rs 50 000 crore liq...,Regulatory benefit bank helping mutual fund in...,"[('Regulatory', 'NNP'), ('benefit', 'NN'), ('b...",14,0,1,3,0.100000,21,20,129,2,3,5.190476,"['bank helping', 'mutual fund']","[0, 7]","[0, 6]",0.000000
78,2020-04-30,Covid-19: Average card spend falls 25% as shop...,"With malls and shops closed

In [41]:
t_train['Label_Final'] = t_train.apply(lambda row : format_label(row), axis=1)

In [42]:
t_train['Label_Final'].value_counts()
#Hence we observe that the number of p

1    7836
0    2164
Name: Label_Final, dtype: int64

**Saving the newly created files in the directory**

In [43]:
data_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/turney2.csv')

In [44]:
t_train.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/turney_v1.csv')

In [45]:
data_test.to_csv('/content/drive/MyDrive/Datasets/Indian Financial News Headlines/data/processed/test_v1.csv')